In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
import numpy as np

# Đọc dữ liệu từ file CSV
data = pd.read_csv('D:\\Python\\MachineLearning\\MachineLearning\\week02\\data\\real_estate.csv')
# Show the description of data 
data.describe() 

# Set to training data (x, y) 
y = data['Y house price of unit area'] 
x = data[['X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']]

# Chuyển danh sách thành mảng numpy
x_data = np.array(x)
y_data = np.array(y)

x_train = x_data[:350]
x_val = x_data[350:]
y_train = y_data[:350]
y_val = y_data[350:]

# Xây dựng mô hình hồi quy tuyến tính với 350 dữ liệu đầu
model = LinearRegression().fit(x_train, y_train)
# In ra các hệ số hồi quy
print("Hệ số hồi quy:")
print("Beta0 (Intercept):", model.intercept_)
print("Beta1 (X2 house age):", model.coef_[0])
print("Beta2 (X3 distance to the nearest MRT station):", model.coef_[1])
print("Beta3 (X4 number of convenience stores):", model.coef_[2])
print("Beta4 (X5 latitude):", model.coef_[3])
print("Beta5 (X6 longitude):", model.coef_[4])

# Dự đoán trên dữ liệu validation
y_pred = model.predict(x_val)

# Tính tổng bình phương sai số
mse_val = np.sum((y_val - y_pred) ** 2)

print("Tổng bình phương sai số của dự đoán trên tập validation:", mse_val)

Hệ số hồi quy:
Beta0 (Intercept): -5491.461092743695
Beta1 (X2 house age): -0.278758171097798
Beta2 (X3 distance to the nearest MRT station): -0.0039896469740573585
Beta3 (X4 number of convenience stores): 1.1519089635746236
Beta4 (X5 latitude): 269.18562233001256
Beta5 (X6 longitude): -9.769304193619735
Tổng bình phương sai số của dự đoán trên tập validation: 4228.146942303107


In [6]:
## Trường hợp sử dụng Numpy – Dùng thuật toán HoldHouse để khai triển QR

def qr_householder(A):
    #""" Compute QR decomposition of A using Householder reflection"""
    M = A.shape[0]
    N = A.shape[1]

    # set Q to the identity matrix
    Q = np.identity(M)

    # set R to zero matrix
    R = np.copy(A)

    for n in range(N):
        # vector to transform
        x = A[n:, n]
        k = x.shape[0]

        # compute ro=-sign(x0)||x||
        ro = -np.sign(x[0]) * np.linalg.norm(x)

        # compute the householder vector v
        e = np.zeros(k)
        e[0] = 1
        v = (1 / (x[0] - ro)) * (x - (ro * e))

        # apply v to each column of A to find R
        for i in range(N):
            R[n:, i] = R[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ R[n:, i])

        # apply v to each column of Q
        for i in range(M):
            Q[n:, i] = Q[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ Q[n:, i])

    return Q.transpose(), R

def linear_regression(x_data, y_data):
# """
# This function calculate linear regression base on x_data and y_data
# :param x_data: vector
# :param y_data: vector
# :return: w (regression estimate)
# """

# add column 1
    x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)

    Q, R = qr_householder(x_bars) # QR decomposition
    R_pinv = np.linalg.pinv(R) # calculate inverse matrix of R
    A = np.dot(R_pinv, Q.T) # apply formula

    return np.dot(A, y_data)

In [11]:
w = linear_regression(x_train, y_train) # get result
w = w.T.tolist()
line = ['Intercept', 'X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']
res = list(zip(line, w))
for o in res:
    print("{: >20}: {: >10}".format(*o))
    
# Thêm cột 1 vào x_data
x_val_with_intercept = np.concatenate((np.ones((x_val.shape[0], 1)), x_val), axis=1)

# Tính dự đoán y_pred
y_pred = np.dot(x_val_with_intercept, w)

# Tính tổng bình phương sai số
mse_val = np.sum((y_val - y_pred) ** 2)

print("Tổng bình phương sai số của dự đoán trên tập validation:", mse_val)

           Intercept: -5491.461092719605
        X2 house age: -0.2787581710978604
X3 distance to the nearest MRT station: -0.003989646974053204
X4 number of convenience stores: 1.1519089635760222
         X5 latitude: 269.1856223299775
        X6 longitude: -9.769304193810802
Tổng bình phương sai số của dự đoán trên tập validation: 4228.146942303843
